In [10]:
import json
import requests
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['axes.grid'] = True
import pandas as pd
import seaborn as sns 
import itertools
import collections
import random
import time
pd.set_option('display.max_colwidth', None)

Get info of all metrics implemented

In [13]:
metrics = []
try:
    request = requests.get('http://seprojects.marum.de:1071/fuji/api/v1/metrics',headers={'Content-Type': 'application/json'}, auth=('marvel2020', 'wonderwoman2020'))
    mm = request.json()['metrics']
    for m in mm:
        metrics.append({k: m[k] for k in ('metric_identifier', 'metric_name', 'total_score')})
except requests.exceptions.RequestException as e:
    print(e)
except requests.exceptions.ConnectionError as e1:
    print(e1)

In [14]:
df_metrics = pd.DataFrame.from_dict(metrics)
df_metrics.head()

,metric_identifier,metric_name,total_score
0,FsF-F1-01D,Data is assigned a globally unique identifier.,1
1,FsF-F1-02D,Data is assigned a persistent identifier.,1
2,FsF-F2-01M,"Metadata includes descriptive core elements (creator, title, data identifier, publisher, publication date, summary and keywords) to support data findability.",2
3,FsF-F3-01M,Metadata includes the identifier of the data it describes.,1
4,FsF-F4-01M,Metadata is offered in such a way that it can be retrieved programmatically.,2


Run FAIR assessment for a given identifier

In [16]:
identifier = 'https://doi.org/10.1594/PANGAEA.902843'

In [21]:
%%time
fuji_service = 'http://seprojects.marum.de:1071/fuji/api/v1/evaluate'
req = {'object_identifier':identifier, 'test_debug':True}
r = requests.post(fuji_service, data=json.dumps(req), headers={'Content-Type': 'application/json'}, auth=('marvel2020', 'wonderwoman2020'))
json_data = r.json()

Wall time: 28.4 s


In [23]:
json_data

[{'id': 1,
  'metric_identifier': 'FsF-F1-01D',
  'metric_name': 'Data is assigned a globally unique identifier.',
  'output': {'guid': 'https://doi.org/10.1594/PANGAEA.902843',
   'guid_scheme': 'doi'},
  'score': {'earned': 1, 'total': 1},
  'test_debug': ["INFO: Unique identifier schemes found ['doi', 'url']",
   'INFO: Finalized unique identifier scheme - doi'],
  'test_status': 'pass'},
 {'id': 2,
  'metric_identifier': 'FsF-F1-02D',
  'metric_name': 'Data is assigned a persistent identifier.',
  'output': {'pid': 'https://doi.org/10.1594/PANGAEA.902843',
   'pid_scheme': 'doi',
   'resolvable_status': True,
   'resolved_url': 'https://doi.pangaea.de/10.1594/PANGAEA.902843'},
  'score': {'earned': 1, 'total': 1},
  'test_debug': ['INFO: Persistence identifier scheme - doi',
   'INFO: Retrieving page http://doi.org/10.1594/PANGAEA.902843',
   'INFO: Content negotiation accept=text/html, application/xhtml+xml, status=200',
   'INFO: Found HTML page!',
   'INFO: Object identifier act

In [25]:
results = {}
results['identifier'] = identifier
for d in json_data:
    metric_identifier = d['metric_identifier']
    sub_principle = metric_identifier.split('-')[1]
    principle = sub_principle[0]+'_earned'
    principle_total = sub_principle[0]+'_total'
    if not sub_principle in results:
        results[sub_principle] = d['score']['earned']
    else:
        results[sub_principle] += d['score']['earned']
    if not principle in results:
        results[principle] = d['score']['earned']
    else:
        results[principle] += d['score']['earned']
    if not principle_total in results:
        results[principle_total] = d['score']['total']
    else:
        results[principle_total] += d['score']['total']

In [27]:
results

{'identifier': 'https://doi.org/10.1594/PANGAEA.902843',
 'F1': 2,
 'F_earned': 6,
 'F_total': 7,
 'F2': 1,
 'F3': 1,
 'F4': 2,
 'A1': 1,
 'A_earned': 1,
 'A_total': 1,
 'I1': 2,
 'I_earned': 3,
 'I_total': 4,
 'I3': 1,
 'R1': 2,
 'R_earned': 6,
 'R_total': 9,
 'R1.1': 1,
 'R1.2': 1,
 'R1.3': 2}

In [ ]:
def visualiza(jsn):
    
    return plot